In [1]:
!pip install vosk==0.3.45 flask flask_cors pyngrok
!apt-get install -y libsndfile1
!wget https://alphacephei.com/vosk/models/vosk-model-small-hi-0.22.zip
!unzip vosk-model-small-hi-0.22.zip

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 20.2 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=60333c514db434a32ffd29c000b2e8f08819a200ec724644e4272d71d580439e
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
--2025-04-29 13:41:49--  https://alphacephei.com/vosk/models/vosk-model-small-hi-0.22.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44458845 (42M) [application/zip]
Saving to: ‘vosk-model-small-hi-0.22.zip’

v

In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from vosk import Model, KaldiRecognizer
import wave
import json
from pyngrok import ngrok

In [13]:
!ngrok config add-authtoken 2wPDgUvkvFjtmraTwQLUcQ6tDPv_4YQFExk8HcddoxTNLvehD

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
app = Flask(__name__)
CORS(app)

In [9]:
model = Model("/content/vosk-model-small-hi-0.22")

In [10]:
@app.route('/stthn', methods=['POST'])
def stt():
    audio_file = request.files['audio']
    wf = wave.open(audio_file, 'rb')
    rec = KaldiRecognizer(model, wf.getframerate())
    rec.SetWords(True)

    text = []
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            text.append(res.get('text', ''))

    # Final result
    final_res = json.loads(rec.FinalResult())
    text.append(final_res.get('text', ''))

    return jsonify({'text': ' '.join(text)})


In [14]:
public_url = ngrok.connect(5001)
print(f"🔥 Ngrok tunnel running at: {public_url}")

app.run(port=5001)

🔥 Ngrok tunnel running at: NgrokTunnel: "https://b256-35-243-144-78.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 13:50:26] "POST /stthn HTTP/1.1" 200 -
